# **Practice 2. Recurrent Neural Networks**

- Alejandro Dopico Castro ([alejandro.dopico2@udc.es](mailto:alejandro.dopico2@udc.es)).
- Ana Xiangning Pereira Ezquerro ([ana.ezquerro@udc.es](mailto:ana.ezquerro@udc.es)).

In [2]:
from data import * 
from metric import * 
from keras.layers import * 
from keras.models import Sequential
from keras.optimizers import Adam 

# global parameters 
TEST_RATIO = 0.2
SEQ_LENGTH = 2
BATCH_SIZE = 30
trainData, testData, stdSales, nFeatures = generateTrainTestData("Walmart.csv", TEST_RATIO, SEQ_LENGTH, BATCH_SIZE) 

2024-03-25 14:14:29.113623: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 14:14:29.145718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:14:29.145744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:14:29.146652: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 14:14:29.152116: I tensorflow/core/platform/cpu_feature_guar

[1.29871732e+01 5.71854780e+05 2.69656599e-01 1.88069047e+01
 4.45318248e-01 3.89025337e+01 1.87857218e+00]


2024-03-25 14:14:30.788530: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1053 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6
2024-03-25 14:14:30.789022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21620 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


## Model

In [3]:
model = Sequential([
    Input(shape=(BATCH_SIZE, nFeatures)),
    Bidirectional(LSTM(units=20, activation='relu', return_sequences=True)),
    Bidirectional(LSTM(units=20, activation='relu', return_sequences=True)),
    Dropout(0.1),
    Bidirectional(LSTM(units=20, activation='relu')),
    Dense(10, activation='relu'), Dense(1)
])
optimizer = Adam(1e-4)

class DenormalizedMAE(Metric):
    def __init__(self, std: float, name='dmae', **kwargs):
        super().__init__(name=name, **kwargs)
        self.std = std 
        self.value = self.add_weight(shape=(), initializer='zeros', name='value')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.multiply(y_true, self.std)
        y_pred = tf.multiply(y_pred, self.std)
        result = tf.reduce_mean(tf.abs(tf.subtract(y_true, y_pred)))
        self.value.assign(result)
        
    def result(self):
        return self.value

model.compile(optimizer, 'mse', metrics=[DenormalizedMAE(stdSales)])
model.fit(trainData, epochs=100, validation_data=testData)

Epoch 1/100


2024-03-25 14:14:40.371387: I external/local_xla/xla/service/service.cc:168] XLA service 0x5f0a1a95eb00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-25 14:14:40.371407: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-25 14:14:40.371412: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-25 14:14:40.376085: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-25 14:14:40.389748: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1711372480.469465 3572375 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


180/180 [==============================] - 10s 20ms/step - loss: 0.9972 - dmae: 283769.6250 - val_loss: 0.8726 - val_dmae: 291034.6562
Epoch 2/100
180/180 [==============================] - 3s 15ms/step - loss: 0.9898 - dmae: 293302.1562 - val_loss: 0.8636 - val_dmae: 288842.5312
Epoch 3/100
180/180 [==============================] - 3s 16ms/step - loss: 0.9698 - dmae: 276767.3750 - val_loss: 0.8361 - val_dmae: 279761.9375
Epoch 4/100
180/180 [==============================] - 3s 15ms/step - loss: 0.9097 - dmae: 227166.4688 - val_loss: 0.7516 - val_dmae: 240674.7031
Epoch 5/100
180/180 [==============================] - 3s 15ms/step - loss: 0.7502 - dmae: 186083.8750 - val_loss: 0.5405 - val_dmae: 162473.9219
Epoch 6/100
180/180 [==============================] - 3s 15ms/step - loss: 0.4376 - dmae: 128981.8281 - val_loss: 0.2099 - val_dmae: 57027.0781
Epoch 7/100
180/180 [==============================] - 3s 15ms/step - loss: 0.2487 - dmae: 114950.1719 - val_loss: 0.1060 - val_dmae: 57